# This notebook will aim to use the trained models weights and load it on the inference model to generate predictions and novel sequences

In [61]:
from models.arcs import load_inf_model, GenPhiloText
from utilities.loaders import load_file
from utilities.preprocessors import preprocess, get_chars, map_value_to_index, decode_predictions
import tensorflow as tf

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load raw data

In [62]:
corpus = load_file('./data/notes.txt')
corpus = preprocess(corpus)
chars = get_chars(corpus)
char_to_idx = map_value_to_index(chars)
idx_to_char = map_value_to_index(chars, inverted=True)

In [63]:
char_to_idx.get_vocabulary()

['[UNK]',
 '\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '´',
 'ç',
 'é',
 'ï',
 '—',
 '…']

In [64]:
n_unique = len(char_to_idx.get_vocabulary())

# Declare same hyper params used in training model

In [65]:
emb_dim = 32
n_a = 64
T_x = 100
dense_layers_dims = [64, n_unique] 
batch_size = 128
alpha = 1e-3
lambda_ = 0.8
drop_prob = 0.4
n_epochs =  50

# Declare sample input to build inference model and access .summary()

In [66]:
sample_input = tf.random.uniform(shape=(1, T_x), minval=0, maxval=n_unique - 1, dtype=tf.int32)
sample_h = tf.zeros(shape=(1, n_a))
sample_c = tf.zeros(shape=(1, n_a))

# Redeclare architecture by passing the same hyper params used in training and then load weights

In [67]:
saved_model = GenPhiloText(emb_dim=emb_dim, n_a=n_a, n_unique=n_unique, T_x=T_x, dense_layers_dims=dense_layers_dims)
saved_model([sample_input, sample_h, sample_c])

[<tf.Tensor: shape=(1, 57), dtype=float32, numpy=
 array([[ 0.00014171, -0.00045676,  0.00213956, -0.00467306,  0.00159151,
         -0.00058444,  0.0028975 ,  0.00350154,  0.00114575,  0.0008153 ,
         -0.00100272, -0.002324  ,  0.0005178 , -0.00228142,  0.00234508,
          0.00213883,  0.00297073, -0.00017764,  0.00200035,  0.00137557,
         -0.00207205, -0.00353116, -0.00275317,  0.00071634, -0.00068335,
          0.00147839, -0.00479024, -0.0013869 ,  0.0005006 ,  0.0021539 ,
          0.00379619,  0.00430242,  0.0003938 ,  0.00522323, -0.0014088 ,
          0.00083025, -0.00189182,  0.00079166, -0.00241121, -0.00024686,
          0.0035631 , -0.00311073, -0.00307755,  0.00164127, -0.00045146,
         -0.00341424, -0.00084708, -0.00116411, -0.00049853,  0.00165734,
         -0.00148998,  0.0032158 , -0.00023968,  0.0023413 ,  0.0036556 ,
         -0.00042517,  0.00452798]], dtype=float32)>,
 <tf.Tensor: shape=(1, 57), dtype=float32, numpy=
 array([[-0.00318379, -0.0015683

In [68]:
saved_model.summary()

Model: "gen_philo_text_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 char-emb-layer (Embedding)  multiple                  1824      
                                                                 
 lstm-cell (LSTM)            multiple                  24832     
                                                                 
 dense-layer-0 (Dense)       multiple                  4160      
                                                                 
 dense-layer-1 (Dense)       multiple                  3705      
                                                                 
 norm-layer-0 (BatchNormali  multiple                  0 (unused)
 zation)                                                         
                                                                 
 act-layer-0 (Activation)    multiple                  0         
                                                  

In [70]:
# saved_model.load_weights(filepath='./saved/weights/notes_gen_philo_text_50_297.5109.h5')
saved_model.load_weights(filepath='./saved/weights/notes_gen_philo_text_99_294.0489.h5')

ValueError: Layer count mismatch when loading weights from file. Model expected 4 layers, found 3 saved layers.

# Extract trainable layers

In [ ]:
layers = saved_model.layers
for layer in layers:
    print(layer.name)

char-emb-layer
lstm-cell
dense-layer-0
dense-layer-1
norm-layer-0
act-layer-0
drop-layer-0
reshape-layer


In [71]:
char_emb_layer = saved_model.get_layer('char-emb-layer')
lstm_cell = saved_model.get_layer('lstm-cell')
dense_layers = []
norm_layers = []
for layer in layers:
    if "dense" in layer.name:
        dense_layers.append(saved_model.get_layer(layer.name))
    if "norm" in layer.name:
        norm_layers.append(saved_model.get_layer(layer.name))

In [72]:
dense_layers[0].get_weights()

[array([[ 0.15413581, -0.04585034,  0.13425063, ..., -0.15007344,
          0.07343598, -0.03324175],
        [ 0.03993504,  0.02864444,  0.08242591, ...,  0.00345853,
         -0.18342778, -0.04860049],
        [-0.10983014,  0.17925711, -0.07110113, ..., -0.19169128,
          0.05988459, -0.03804645],
        ...,
        [-0.16686128,  0.19760524, -0.00297822, ...,  0.20005806,
         -0.11622456,  0.2137296 ],
        [ 0.04940142, -0.07836404,  0.19478305, ...,  0.01661502,
          0.0962223 ,  0.01452222],
        [ 0.08875827,  0.20963843,  0.13392507, ..., -0.1728583 ,
         -0.19741769,  0.01224127]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)]

In [73]:
len(char_emb_layer.get_weights())

1

In [74]:
lstm_cell.get_weights()

[array([[ 0.09164128, -0.12331849,  0.0038289 , ...,  0.12636712,
          0.11929837, -0.02396945],
        [ 0.12735015, -0.05223182, -0.07983956, ...,  0.10046722,
          0.00435773,  0.14000514],
        [-0.03871424,  0.06427422,  0.11393535, ..., -0.12418804,
          0.08219762,  0.05787762],
        ...,
        [ 0.1331704 , -0.02678926,  0.07699445, ...,  0.03459013,
          0.10233064, -0.05794905],
        [-0.14056163, -0.09337792,  0.02954078, ..., -0.06918932,
         -0.06614012, -0.11815637],
        [ 0.13767329,  0.05735423, -0.14110614, ...,  0.05787472,
         -0.01346493, -0.04260255]], dtype=float32),
 array([[ 0.08775997,  0.00374378,  0.02353095, ..., -0.01370538,
          0.08408511, -0.07424256],
        [ 0.06199227, -0.04240251, -0.00318493, ...,  0.07032265,
          0.11686444,  0.01087513],
        [-0.05497014,  0.0670696 ,  0.01998818, ...,  0.10044608,
         -0.01404738, -0.01179312],
        ...,
        [-0.06380627, -0.04370848, -0.0

# Build inference model

In [79]:
inference_model = load_inf_model(char_emb_layer, lstm_cell, dense_layers, char_to_idx=char_to_idx, T_x=250, temperature=1.2)

57
64
tf.Tensor(
[[-inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]], shape=(1, 57), dtype=float32)
250
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
1

In [80]:
inference_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 char-emb-layer (Embedding)  (None, 1, 32)                1824      ['input_6[0][0]',             
                                                                     'tf.random.categorical_1000[0
                                                                    ][0]',                        
                                                                     'tf.random.categorical_1001[0
                                                                    ][0]',                        
                                                                     'tf.random.categorical_

In [81]:
pred_ids = inference_model.predict([sample_input, sample_h, sample_c])
pred_ids

1/1 [==============================] - 126s 126s/step


[array([[45]], dtype=int64),
 array([[27]], dtype=int64),
 array([[39]], dtype=int64),
 array([[55]], dtype=int64),
 array([[17]], dtype=int64),
 array([[50]], dtype=int64),
 array([[15]], dtype=int64),
 array([[13]], dtype=int64),
 array([[55]], dtype=int64),
 array([[37]], dtype=int64),
 array([[24]], dtype=int64),
 array([[55]], dtype=int64),
 array([[7]], dtype=int64),
 array([[52]], dtype=int64),
 array([[25]], dtype=int64),
 array([[25]], dtype=int64),
 array([[1]], dtype=int64),
 array([[51]], dtype=int64),
 array([[27]], dtype=int64),
 array([[15]], dtype=int64),
 array([[43]], dtype=int64),
 array([[5]], dtype=int64),
 array([[55]], dtype=int64),
 array([[17]], dtype=int64),
 array([[9]], dtype=int64),
 array([[18]], dtype=int64),
 array([[27]], dtype=int64),
 array([[9]], dtype=int64),
 array([[37]], dtype=int64),
 array([[40]], dtype=int64),
 array([[51]], dtype=int64),
 array([[9]], dtype=int64),
 array([[17]], dtype=int64),
 array([[34]], dtype=int64),
 array([[6]], dtype=

In [82]:
decode_predictions(pred_ids, idx_to_char)

'uco—4z20—m?—(çaa\n´c2s&—4,6c,mp´,4j\'7j67feg,-3g.osu(fp;7b[UNK]!ées"exz?fpxhzm2v-n\nh:2f71de0r,!d—k."9.6rç-,.j8h6iç0?49yyey:9 4ç.ad6\')1…a,:ufjz/fxx7)bq4bnet—0çy1,3gé?[UNK]gr7éï2—2b;hwl02 2!ct9[UNK]ueyzc6y(\'yï"ï.ïh?i?/t&cl,p6kp:)l\'y…b1?x-3"y?xfj\'j.&iéfcç6i:ïao/?x3,4'